# P5: Vehicle Detection and Tracking

## Import the necessary libraries

In [1]:
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import glob
import time
from random import shuffle
from scipy.ndimage.measurements import label
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from moviepy.editor import VideoFileClip

/Users/Claudio/anaconda/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Useful functions to be used in the image pipeline for Vehicle Detection and Tracking

In [2]:
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'BGR2LUV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
    if conv == 'RGB2HSV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    if conv == 'BGR2HSV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    if conv == 'RGB2HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if conv == 'BGR2HLS':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    if conv == 'RGB2YUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    if conv == 'BGR2YUV':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    if conv == 'RGB2BGR':
        return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    if conv == 'BGR2RGB':
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [3]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_color(img_tosearch, conv='RGB2HLS')
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell)-1
    nyblocks = (ch1.shape[0] // pix_per_cell)-1 
    nfeat_per_block = orient*cell_per_block**2
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell)-1 
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    boxes_list = []
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6)
                box = np.array([[xbox_left,ytop_draw+ystart],[xbox_left+win_draw,ytop_draw+win_draw+ystart]])
                boxes_list.append(box)
    return (draw_img,boxes_list)

In [4]:
## Heat-map functions
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

## Training

In [5]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial(), color_hist() and get_hog_features()
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins)
        # Append the new feature vector to the features list
        combined_features = np.concatenate((spatial_features, hist_features))
        combined_features = np.concatenate((combined_features, hog_features))
        # Append the new feature vector to the features list
        features.append(combined_features)
        
    # Return list of feature vectors
    return features

In [6]:
def process_glob_images(images, category):
    for image in images:
        if category == 'cars':
            cars.append(image)
        else:
            notcars.append(image)

In [7]:
# Divide up into cars and notcars
# Read in car and non-car images
cars = []
car_images1 = glob.glob('training_data/vehicles/GTI_Far/*.png')
car_images2 = glob.glob('training_data/vehicles/GTI_Left/*.png')
car_images3 = glob.glob('training_data/vehicles/GTI_MiddleClose/*.png')
car_images4 = glob.glob('training_data/vehicles/GTI_Right/*.png')
car_images5 = glob.glob('training_data/vehicles/KITTI_extracted/*.png')
process_glob_images(car_images1,'cars')
process_glob_images(car_images1,'cars')
process_glob_images(car_images1,'cars')
process_glob_images(car_images1,'cars')
process_glob_images(car_images1,'cars')
print("There are " + str(len(cars)) + " cars images in the training dataset")
shuffle(cars)

notcars = []
non_car_images1 = glob.glob('training_data/non-vehicles/Extras/*.png')
non_car_images2 = glob.glob('training_data/non-vehicles/GTI/*.png')
process_glob_images(non_car_images1,'notcars')
process_glob_images(non_car_images2,'notcars')
shuffle(notcars)
#notcars = notcars[:len(cars)]
print("There are " + str(len(notcars)) + " not-cars images in the training dataset")


colorspace = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb       
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions 
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [400, 656] # Min and max in y to search in slide_window()

t=time.time()
car_features = extract_features(cars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

There are 4170 cars images in the training dataset
There are 8968 not-cars images in the training dataset
80.72 Seconds to extract HOG features...
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8412
34.49 Seconds to train SVC...
Test Accuracy of SVC =  0.997
My SVC predicts:  [ 0.  0.  1.  0.  0.  0.  0.  1.  0.  1.]
For these 10 labels:  [ 0.  0.  1.  0.  0.  0.  0.  1.  0.  1.]
0.01996 Seconds to predict 10 labels with SVC


## Image Pipeline

In [8]:
# This function makes sure that each processed image is saved in the 
# appropriate folder 
def save_img(img, folder, fname, stage_name, col_map):
    if fname != "None":
        if not os.path.exists(folder):
            os.makedirs(folder)
        if fname != "chosen":
            fname = fname.split('/')[1]
            fname = fname.split('.')[0]
            new_filename = fname + "_" + stage_name + '.jpg'
        else:
            new_filename = stage_name + '.jpg'
            
        mpimg.imsave(folder + "/" + new_filename, img,cmap=col_map)

In [9]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    draw_img = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(draw_img, (bbox[0][0],bbox[0][1]), (bbox[1][0],bbox[1][1]), color, thick)
    # Return the image copy with boxes drawn
    return draw_img

In [10]:
ystart = y_start_stop[0]   
ystop = y_start_stop[1]

# This function processes each individual image coming from the video stream 
# and estimates where the cars are
def image_pipeline(img, fname):
    out_img, boxes_list_1 = find_cars(img, y_start_stop[0] , y_start_stop[1], 1.5, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    out_img, boxes_list_2 = find_cars(img, y_start_stop[0] , y_start_stop[1], 2, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    out_img, boxes_list_3 = find_cars(img, y_start_stop[0] , y_start_stop[1], 1, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    
    boxes_list = boxes_list_1 + boxes_list_2 + boxes_list_3
    result = draw_boxes(img, tuple(boxes_list))
    save_img(result, "output_images/pre_heat_map", fname, "pre_heat_map", col_map = 'jet')
    
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    
    # Add heat to each box in box list
    heat = add_heat(heat,boxes_list)  
    save_img(heat, "output_images/heat_pre_threshold", fname, "heat_pre_threshold", col_map = 'hot')
    
    global heat_previous
    if fname == "None":
        heat_previous = heat_previous*0.6
        heat_previous = heat_previous + heat 
        heat = apply_threshold(heat_previous,4)

    # Apply threshold to help remove false positives
    if fname != "None":
        heat = apply_threshold(heat,2)
    save_img(heat, "output_images/heat_post_threshold", fname, "heat_post_threshold", col_map = 'gray')
    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    heat_img = draw_labeled_bboxes(np.copy(img), labels)
    save_img(heat_img, "output_images/post_heat_map", fname, "post_heat_map", col_map = 'jet')
    return heat_img

## Test Images

In [11]:
# Create the output_images directory 
if not os.path.exists("output_images"):
    os.makedirs("output_images")

# Read in and make a list of the test images
test_images = glob.glob('test_images/*.jpg')

for fname in test_images:
    # read in each image
    img = mpimg.imread(fname)
    
    result = image_pipeline(img, fname)

In [12]:
# Show Hog Features on a couple of training images

fig1 = plt.figure()
# Car Image
car_image = mpimg.imread(cars[0])
car_image_HLS = convert_color(car_image, conv='RGB2HLS')
plt.subplot(221)
plt.imshow(car_image)
plt.title('Car (RGB)')
plt.subplot(223)
plt.imshow(car_image_HLS)
plt.title('Car (HLS)')
# Non-Car Image
non_car_img = mpimg.imread(notcars[0])
non_car_image_HLS = convert_color(non_car_img, conv='RGB2HLS')
plt.subplot(222)
plt.imshow(non_car_img)
plt.title('Not-Car (RGB)')
plt.subplot(224)
plt.imshow(non_car_image_HLS)
plt.title('Not-Car (HLS)')
plt.tight_layout()
fig1.savefig('training_examples/car_and_noncar.png')


fig2 = plt.figure()
# Car channels and HOG features 
ch1_car = car_image_HLS[:,:,0]
ch2_car = car_image_HLS[:,:,1]
ch3_car = car_image_HLS[:,:,2]
(features_ch1_car, hog_image_ch1_car) = get_hog_features(ch1_car, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
(features_ch2_car, hog_image_ch2_car) = get_hog_features(ch2_car, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
(features_ch3_car, hog_image_ch3_car) = get_hog_features(ch3_car, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
# Non-car channels and HOG features
ch1_non_car = non_car_image_HLS[:,:,0]
ch2_non_car = non_car_image_HLS[:,:,1]
ch3_non_car = non_car_image_HLS[:,:,2]
(features_ch1_non_car, hog_image_ch1_non_car) = get_hog_features(ch1_non_car, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
(features_ch2_non_car, hog_image_ch2_non_car) = get_hog_features(ch2_non_car, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
(features_ch3_non_car, hog_image_ch3_non_car) = get_hog_features(ch3_non_car, orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)

plt.subplot2grid((3,4), (0,0))
plt.imshow(ch1_car, cmap='gray')
plt.title('Car ch1')
plt.subplot2grid((3,4), (1,0))
plt.imshow(ch2_car, cmap='gray')
plt.title('Car ch2')
plt.subplot2grid((3,4), (2,0))
plt.imshow(ch3_car, cmap='gray')
plt.title('Car ch3')

plt.subplot2grid((3,4), (0,1))
plt.imshow(hog_image_ch1_car, cmap='gray')
plt.title('Car ch1 Hog')
plt.subplot2grid((3,4), (1,1))
plt.imshow(hog_image_ch2_car, cmap='gray')
plt.title('Car ch2 Hog')
plt.subplot2grid((3,4), (2,1))
plt.imshow(hog_image_ch3_car, cmap='gray')
plt.title('Car ch3 Hog')

plt.subplot2grid((3,4), (0,2))
plt.imshow(ch1_non_car, cmap='gray')
plt.title('Non-Car ch1')
plt.subplot2grid((3,4), (1,2))
plt.imshow(ch2_non_car, cmap='gray')
plt.title('Non-Car ch2')
plt.subplot2grid((3,4), (2,2))
plt.imshow(ch3_non_car, cmap='gray')
plt.title('Non-Car ch3')

plt.subplot2grid((3,4), (0,3))
plt.imshow(hog_image_ch1_non_car, cmap='gray')
plt.title('Non-Car ch1 Hog')
plt.subplot2grid((3,4), (1,3))
plt.imshow(hog_image_ch2_non_car, cmap='gray')
plt.title('Non-Car ch2 Hog')
plt.subplot2grid((3,4), (2,3))
plt.imshow(hog_image_ch3_non_car, cmap='gray')
plt.title('Non-Car ch3 Hog')

plt.tight_layout()
fig2.savefig('training_examples/HOG_features.png')


## Test on Videos

In [13]:
heat_previous = np.zeros_like(img[:,:,0]).astype(np.float)

In [14]:
def process_image(image):
    result = image_pipeline(image, "None")
    return result

In [15]:
test_output = 'test_video_output.mp4'
clip1 = VideoFileClip("test_video.mp4")
test_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time test_clip.write_videofile(test_output, audio=False)

[MoviePy] >>>> Building video test_video_output.mp4
[MoviePy] Writing video test_video_output.mp4


 97%|█████████▋| 38/39 [01:01<00:01,  1.48s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_output.mp4 

CPU times: user 57.7 s, sys: 2.66 s, total: 1min
Wall time: 1min 2s


In [16]:
heat_previous = np.zeros_like(img[:,:,0]).astype(np.float)

In [17]:
project_output = 'project_video_output.mp4'
clip2 = VideoFileClip("project_video.mp4")
project_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [35:13<00:01,  1.60s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 33min, sys: 1min 46s, total: 34min 46s
Wall time: 35min 14s
